<a href="https://colab.research.google.com/github/easywater4528/my-ML-project/blob/main/mini_proj_04_20221322.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor

# 점수 출력 함수 포맷 설정
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())


In [9]:
df = pd.read_csv("winequality-red.csv")

print("데이터 크기:", df.shape)
df.head()
df.describe()

데이터 크기: (1599, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [10]:
X = df.drop("quality", axis=1)
y = df["quality"]

print("X shape:", X.shape)
print("y shape:", y.shape)

#교차검증
cv = KFold(n_splits=10, shuffle=True, random_state=42)

X shape: (1599, 11)
y shape: (1599,)


In [12]:
#랜덤 포레스트 사용
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# 하이퍼파라미터
param_dist_rf = {
    "n_estimators": [200, 300, 500, 800],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 4, 6, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.5, 0.7],
    "bootstrap": [True, False]
}

search_rf = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist_rf,
    n_iter=40,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    random_state=42
)

search_rf.fit(X, y)

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
                   estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
                   n_iter=40, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2', 0.5,
                                                         0.7],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 4, 6, 10],
                                        'n_estimators': [200, 300, 500, 800]},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [ ]:
# 2차 튜닝
base_n_estimators = search_rf.best_params_["n_estimators"]
base_max_depth = search_rf.best_params_["max_depth"]
base_min_split = search_rf.best_params_["min_samples_split"]
base_min_leaf = search_rf.best_params_["min_samples_leaf"]
base_max_features = search_rf.best_params_["max_features"]
base_bootstrap = search_rf.best_params_["bootstrap"]


In [ ]:
# GridSearchCV
param_grid_rf_fine = {
    "n_estimators": [
        max(200, base_n_estimators - 200),
        base_n_estimators,
        base_n_estimators + 200
    ],
    "max_depth": (
        [None] if base_max_depth is None else
        [max(1, base_max_depth - 5), base_max_depth, base_max_depth + 5]
    ),
    "min_samples_split": [
        max(2, base_min_split - 2),
        base_min_split,
        base_min_split + 2
    ],
    "min_samples_leaf": [
        max(1, base_min_leaf - 1),
        base_min_leaf,
        base_min_leaf + 1
    ],
    "max_features": [base_max_features],
    "bootstrap": [base_bootstrap]
}

rf_fine = RandomForestRegressor(random_state=42, n_jobs=-1)

grid_rf_fine = GridSearchCV(
    estimator=rf_fine,
    param_grid=param_grid_rf_fine,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

grid_rf_fine.fit(X, y)

best_rf_final = grid_rf_fine.best_estimator_
best_rmse_final = -grid_rf_fine.best_score_

# 최종 교차검증
neg_rmse_scores_final = cross_val_score(
    best_rf_final,
    X, y,
    scoring="neg_root_mean_squared_error",
    cv=cv
)

rmse_scores_final = -neg_rmse_scores_final

print("\n[RandomForest - 최종 튜닝 모델 결과 값]")
display_scores(rmse_scores_final)



[RandomForest - 최종 튜닝 모델 결과 값]
Scores: [0.54238186 0.51964678 0.58905601 0.52894379 0.56334497 0.58888149
 0.55785222 0.59563075 0.52320269 0.52747654]
Mean: 0.553641709905199
Standard deviation: 0.02806163352232548
